In [ ]:
import urllib.request
from bs4 import BeautifulSoup

# 접근할 페이지 번호
pageNum = 2

# 저장할 이미지 경로 및 이름
imageNum = 0
imageDir = "/content/drive/MyDrive/TON2/sunset/k_"

while pageNum < 320: # 필요한 페이지에 맞추기
  url1 = "https://wallpaperscraft.com/tag/sunset/page"
  # url2 = "&query=sunset&size="
  url = url1 + str(pageNum) #+ url2

  fp = urllib.request.urlopen(url)
  source = fp.read();
  fp.close()

  soup = BeautifulSoup(source, "html.parser")
  soup = soup.findAll("span", class_ = "wallpapers__canvas")

  # 이미지 경로 받아 로컬에 저장
  for i in soup:
    imageNum += 1
    imgURL = i.find("img")["src"]
    urllib.request.urlretrieve(imgURL, imageDir + str(imageNum).zfill(4) + ".jpg")
    # urllib.request.urlretrieve("https:" + imgURL, imageDir + str(imageNum).zfill(4) + ".jpg") # https: 없을때
    print(imgURL)
    print(imageNum)

  pageNum += 1

https://images.wallpaperscraft.com/image/single/sunset_dark_twilight_148787_300x168.jpg
1
https://images.wallpaperscraft.com/image/single/sunset_clouds_sun_144022_300x168.jpg
2
https://images.wallpaperscraft.com/image/single/sunset_lake_grass_trees_109375_300x168.jpg
3
https://images.wallpaperscraft.com/image/single/sunset_sky_clouds_light_86141_300x168.jpg
4
https://images.wallpaperscraft.com/image/single/sunset_sea_horizon_84629_300x168.jpg
5
https://images.wallpaperscraft.com/image/single/sunset_river_trees_landscape_86149_300x168.jpg
6
https://images.wallpaperscraft.com/image/single/sunset_path_trees_159376_300x168.jpg
7
https://images.wallpaperscraft.com/image/single/sunset_sea_waves_145862_300x168.jpg
8
https://images.wallpaperscraft.com/image/single/sunset_nature_beach_water_light_ocean_sun_77620_300x168.jpg
9
https://images.wallpaperscraft.com/image/single/sunset_sea_beach_landscape_light_86146_300x168.jpg
10
https://images.wallpaperscraft.com/image/single/sunset_road_hills_sky

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://images.wallpaperscraft.com/image/single/sunset_architecture_mosque_123964_300x168.jpg
16
https://images.wallpaperscraft.com/image/single/sunset_sea_buildings_123331_300x168.jpg
17
https://images.wallpaperscraft.com/image/single/sunset_trees_nature_landscape_86140_300x168.jpg
18
https://images.wallpaperscraft.com/image/single/sunset_trees_sky_107304_300x168.jpg
19
https://images.wallpaperscraft.com/image/single/sunset_trees_sky_night_107906_300x168.jpg
20
https://images.wallpaperscraft.com/image/single/sunset_horizon_pier_183300_300x168.jpg
21
https://images.wallpaperscraft.com/image/single/sunset_sun_dark_161572_300x168.jpg
22
https://images.wallpaperscraft.com/image/single/sunset_sea_horizon_157669_300x168.jpg
23
https://images.wallpaperscraft.com/image/single/sunset_fog_tree_147343_300x168.jpg
24
https://images.wallpaperscraft.com/image/single/sunset_sea_horizon_143364_300x168.jpg
25
https://images.wallpaperscraft.com/image/single/sunset_landscape_art_127678_300x168.jpg
26
ht

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://images.wallpaperscraft.com/image/single/sunset_autumn_river_trees_118215_300x168.jpg
31
https://images.wallpaperscraft.com/image/single/sunset_sky_night_23686_300x168.jpg
32
https://images.wallpaperscraft.com/image/single/sunset_sky_sea_clouds_sand_wet_mirror_reflection_48121_300x168.jpg
33
https://images.wallpaperscraft.com/image/single/sunset_sun_rays_201655_300x168.jpg
34
https://images.wallpaperscraft.com/image/single/sunset_trees_snow_195038_300x168.jpg
35
https://images.wallpaperscraft.com/image/single/sunset_mountains_grass_192379_300x168.jpg
36
https://images.wallpaperscraft.com/image/single/sunset_sun_birds_192167_300x168.jpg
37
https://images.wallpaperscraft.com/image/single/sunset_horizon_sea_187903_300x168.jpg
38
https://images.wallpaperscraft.com/image/single/sunset_coast_grass_152281_300x168.jpg
39
https://images.wallpaperscraft.com/image/single/sunset_sea_waves_146016_300x168.jpg
40
https://images.wallpaperscraft.com/image/single/sunset_sun_branches_145600_300x16

...

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://images.wallpaperscraft.com/image/single/sunset_sun_mountains_205512_300x168.jpg
361
https://images.wallpaperscraft.com/image/single/sunset_sun_water_205212_300x168.jpg
362
https://images.wallpaperscraft.com/image/single/sunset_water_city_204993_300x168.jpg
363
https://images.wallpaperscraft.com/image/single/sun_clouds_sea_202122_300x168.jpg
364
https://images.wallpaperscraft.com/image/single/sunset_sea_shore_201119_300x168.jpg
365
https://images.wallpaperscraft.com/image/single/sunset_sun_mountains_200764_300x168.jpg
366
https://images.wallpaperscraft.com/image/single/sunset_sea_beach_196263_300x168.jpg
367
https://images.wallpaperscraft.com/image/single/sunset_sun_clouds_196150_300x168.jpg
368
https://images.wallpaperscraft.com/image/single/sunset_sea_rocks_195843_300x168.jpg
369
https://images.wallpaperscraft.com/image/single/sunset_clouds_rays_195828_300x168.jpg
370
https://images.wallpaperscraft.com/image/single/sunset_city_buildings_195639_300x168.jpg
371
https://images.wa

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://images.wallpaperscraft.com/image/single/palm_road_sunset_110515_300x168.jpg
1816
https://images.wallpaperscraft.com/image/single/sea_shore_sunset_sand_117848_300x168.jpg
1817
https://images.wallpaperscraft.com/image/single/sky_clouds_nature_254547_300x168.jpg
1818
https://images.wallpaperscraft.com/image/single/ship_horizon_sunset_172147_300x168.jpg
1819
https://images.wallpaperscraft.com/image/single/mountains_lake_sunset_150248_300x168.jpg
1820
https://images.wallpaperscraft.com/image/single/rock_sea_sunset_132241_300x168.jpg
1821
https://images.wallpaperscraft.com/image/single/mountains_lake_sunset_128485_300x168.jpg
1822
https://images.wallpaperscraft.com/image/single/field_path_sunset_126997_300x168.jpg
1823
https://images.wallpaperscraft.com/image/single/mountain_sky_sunset_125061_300x168.jpg
1824
https://images.wallpaperscraft.com/image/single/chicago_skyscrapers_bridges_120484_300x168.jpg
1825
https://images.wallpaperscraft.com/image/single/sea_sun_sunset_ripples_92636_

KeyboardInterrupt: ignored

In [ ]:


import urllib.request
from bs4 import BeautifulSoup

# 접근할 페이지 번호
pageNum = 2

# 저장할 이미지 경로 및 이름
imageNum = 
imageDir = "/content/drive/MyDrive/TON2/sunset/k_"

while pageNum < 320: # 필요한 페이지에 맞추기
  url1 = "http://wallpaperswide.com/search/page/2?q=sunset"
  url2 = "?q=sunset"
  url = url1 + str(pageNum) + url2

  fp = urllib.request.urlopen(url)
  source = fp.read();
  fp.close()

  soup = BeautifulSoup(source, "html.parser")
  soup = soup.findAll("div", class_ = "thumb")

  # 이미지 경로 받아 로컬에 저장
  for i in soup:
    imageNum += 1
    imgURL = i.find("img")["src"]
    urllib.request.urlretrieve(imgURL, imageDir + str(imageNum).zfill(4) + ".jpg")
    # urllib.request.urlretrieve("https:" + imgURL, imageDir + str(imageNum).zfill(4) + ".jpg") # https: 없을때
    print(imgURL)
    print(imageNum)

  pageNum += 1

In [ ]:
!pip install selenium

     |████████████████████████████████| 979 kB 5.2 MB/s 
     |████████████████████████████████| 138 kB 51.8 MB/s 
     |████████████████████████████████| 359 kB 62.2 MB/s 
     |████████████████████████████████| 4.0 MB 37.2 MB/s 
     |████████████████████████████████| 55 kB 5.0 MB/s 
     |████████████████████████████████| 58 kB 7.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
from multiprocessing import Pool 
from bs4 import BeautifulSoup
import urllib.request
import requests
import time
import os 

image_num = 0
img_url = []
# img_folder = './sunset'
# if not os.path.isdir(img_folder) : 
#     os.mkdir(img_folder)
imageDir = "/content/drive/MyDrive/TON2/sunset/"

def get_sublist_href():
    links = []
    pageNum = 1
    while pageNum<320:
        url1 = "https://wallpaperscraft.com/search/?order=&page="
        url2 = "&query=sunset&size="
        url = url1 + str(pageNum) + url2
        
        response = requests.get(url)
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        ul = soup.select_one('ul.wallpapers__list')

        data = ul.select('li > a')
        for d in data:
            links.append(d.attrs['href'])
        
        pageNum+=1
    return links

def do_html_crawl(link):
    global image_num
    abs_link = 'https://wallpaperscraft.com'+link
    req = requests.get(abs_link)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    try:
        img_url = soup.select_one('img.wallpaper__image')['src']
    except TypeError:
        import pdb;pdb.set_trace()
    urllib.request.urlretrieve(img_url, imageDir + str(image_num).zfill(4) + '.jpg')
    image_num+=1

def do_thread_crawl(urls):
    thread_list = []
    with ThreadPoolExecutor(max_workers=4) as executor:
        for url in urls:
            thread_list.append(executor.submit(do_html_crawl, url))
        for execution in concurrent.futures.as_completed(thread_list):
            execution.result()
    
    
if __name__ == '__main__':
    start_time = time.time()
    do_thread_crawl(get_sublist_href())
    print(f"--- {(time.time() - start_time):.2f}seconds ---")

KeyboardInterrupt: ignored